In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

import matplotlib.pyplot as plt
%matplotlib inline


     |████████████████████████████████| 1.7MB 10.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=d2975ecb326f43658091b92ee2a6db9613f35a35fcb18ed1a80f5965e37a01e3
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
stopwords = set(nltk_stopwords.words('russian'))

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect("/content/drive/MyDrive/anekdot-ru.db")
cursor = conn.cursor()
df_anek = pd.read_sql_query("SELECT text FROM anecdot WHERE date BETWEEN '2019-01-01' and '2019-01-31'", conn)

In [ ]:
df_anek

,text
0,1 января — День святого рассола и минералки.
1,"Уровень тишины 1 января: слышно, как у соседей..."
2,В 8 часов утра 1 января собачники могли бы зах...
3,"- А я вот загадал на Новый Год желание, чтобы ..."
4,Новый год без ели – что Дед Мороз без хмеля!\n...
...,...
1995,"Это как надо было не хотеть дочку, чтобы назва..."
1996,"Всё было отлично, но потом в комнату зашёл экс..."
1997,Хитрожопый краб торговал своими клешнями. Это ...
1998,"- А ведь я полюбил тебя, дорогая, за твою кули..."


In [ ]:
def preprocess(data):
  data_tokenized = []
  for text in data:
    text_c = text.lower()
    text_cle = re.findall(r'\w+', text_c)
    text_clean = [w for w in text_cle if not w in stopwords]
    data_tokenized.append(text_clean)

  return data_tokenized

anekdots = preprocess(df_anek['text'])

In [ ]:
anekdots[:2]

[['1', 'января', 'день', 'святого', 'рассола', 'минералки'],
 ['уровень',
  'тишины',
  '1',
  'января',
  'слышно',
  'соседей',
  'сверху',
  'вибрирует',
  'телефон']]

In [ ]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
anek_lem = []
for t in anekdots:
  sent = []
  for w in t:
    if w.isdigit() is False:
      sent.append(morph.parse(w)[0].normal_form)
  anek_lem.append(sent)

In [ ]:
anek_lem[:2]

[['январь', 'день', 'святой', 'рассол', 'минералка'],
 ['уровень',
  'тишина',
  'январь',
  'слышный',
  'сосед',
  'сверху',
  'вибрировать',
  'телефон']]

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(anek_lem)
# Create Corpus
texts = anek_lem
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:2])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]]


In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('день', 1), ('минералка', 1), ('рассол', 1), ('святой', 1), ('январь', 1)]]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30, 
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
for t, top_words in lda_model.print_topics(num_topics=50, num_words=10):
  print('Topic', t+1, ':', top_words)

Topic 1 : 0.014*"это" + 0.009*"писать" + 0.009*"год" + 0.008*"письмо" + 0.007*"человек" + 0.006*"мужик" + 0.006*"свой" + 0.005*"дедушка" + 0.005*"россия" + 0.005*"секс"
Topic 2 : 0.013*"это" + 0.005*"человек" + 0.005*"похожий" + 0.005*"ранний" + 0.004*"сделать" + 0.004*"новый" + 0.004*"год" + 0.004*"ещё" + 0.004*"вместо" + 0.004*"почему"
Topic 3 : 0.009*"свой" + 0.009*"всё" + 0.007*"это" + 0.006*"который" + 0.005*"работать" + 0.005*"один" + 0.005*"новый" + 0.005*"водка" + 0.005*"любить" + 0.004*"путин"
Topic 4 : 0.014*"это" + 0.007*"который" + 0.007*"человек" + 0.006*"почему" + 0.006*"страна" + 0.006*"ввести" + 0.006*"венесуэла" + 0.005*"знать" + 0.005*"власть" + 0.005*"против"
Topic 5 : 0.010*"это" + 0.006*"путин" + 0.005*"человек" + 0.005*"украина" + 0.005*"делать" + 0.005*"два" + 0.005*"знать" + 0.004*"россия" + 0.004*"мой" + 0.004*"свой"
Topic 6 : 0.010*"новый" + 0.009*"год" + 0.007*"россия" + 0.005*"япония" + 0.005*"время" + 0.005*"говорить" + 0.005*"маленький" + 0.004*"экономика"

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=anek_lem, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Perplexity:  -9.259945161246456
# Coherence Score:  0.4773445677408298

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
26    -0.031763  0.072729       1        1  5.060912
23     0.242347  0.039761       2        1  4.643702
8     -0.037449  0.073883       3        1  4.279185
20     0.042805  0.033710       4        1  4.080855
5     -0.013051  0.060781       5        1  3.982350
18     0.000549  0.078901       6        1  3.725939
2      0.006682 -0.071548       7        1  3.618305
19    -0.006716 -0.057219       8        1  3.615633
3      0.029238  0.010602       9        1  3.580651
9     -0.027122  0.004399      10        1  3.555319
4     -0.065822  0.002580      11        1  3.476423
22     0.019406 -0.045360      12        1  3.465552
0     -0.029251  0.071011      13        1  3.398915
24     0.018828 -0.115582      14        1  3.312386
1     -0.068941 -0.004720      15        1  3.281358
16    -0.068655 -0.019443      16        1  3.256780
17     0.019550  0.092779      17        1  3.248675
6      0.007425 -0.004042      18        1  3.197020
12     0.035479 -0.075377      19        1  3.117641
7     -0.016368 -0.044965      20        1  3.107471
10     0.003471 -0.017561      21        1  3.042812
15    -0.010200  0.046586      22        1  2.915227
14    -0.019361 -0.057622      23        1  2.906930
27    -0.033445  0.001252      24        1  2.838742
25     0.023718 -0.054892      25        1  2.646957
13    -0.020350 -0.032231      26        1  2.630220
11     0.012905 -0.053305      27        1  2.553146
28    -0.027452  0.029855      28        1  2.541163
29    -0.010110  0.022631      29        1  2.509662
21     0.023652  0.012406      30        1  2.410072, topic_info=         Term        Freq       Total Category  logprob  loglift
1548     яйцо   73.000000   73.000000  Default  30.0000  30.0000
144   мужчина   47.000000   47.000000  Default  29.0000  29.0000
142   женщина   54.000000   54.000000  Default  28.0000  28.0000
30        это  247.000000  247.000000  Default  27.0000  27.0000
294     путин   58.000000   58.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
617    первый    2.117336   45.210289  Topic30  -5.7707   0.6643
12      время    2.117181   56.091381  Topic30  -5.7708   0.4486
428      один    2.117519   63.575245  Topic30  -5.7706   0.3235
1548     яйцо    2.117216   73.236143  Topic30  -5.7708   0.1819
51    человек    1.941837  107.708089  Topic30  -5.8572  -0.2903

[2217 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1812     21  0.667294  aмeриканский
1602      9  0.654229    bellingcat
4414      1  0.244095             c
4414      8  0.081365             c
4414     11  0.081365             c
...     ...       ...           ...
2818      9  0.657778          ясно
2818     22  0.328889          ясно
5328     21  0.891970         ясный
992       1  0.324724        ёлочка
992      19  0.649448        ёлочка

[6284 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[27, 24, 9, 21, 6, 19, 3, 20, 4, 10, 5, 23, 1, 25, 2, 17, 18, 7, 13, 8, 11, 16, 15, 28, 26, 14, 12, 29, 30, 22])

In [ ]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=anek_lem, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
num_of_docs = len(corpus)
num_of_docs
type(len(corpus))

int

In [ ]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

In [ ]:
model_results['Alpha']
model_results['Beta']
model_results['Coherence']

In [ ]:
result = pd.read_csv('/content/lda_tuning_results.csv')
result = pd.DataFrame(result)
result

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.185373
1,75% Corpus,2,0.01,0.31,0.220443
2,75% Corpus,2,0.01,0.61,0.207812
3,75% Corpus,2,0.01,0.9099999999999999,0.178013
4,75% Corpus,2,0.01,symmetric,0.187350
...,...,...,...,...,...
535,100% Corpus,10,asymmetric,0.01,0.468723
536,100% Corpus,10,asymmetric,0.31,0.557989
537,100% Corpus,10,asymmetric,0.61,0.463407
538,100% Corpus,10,asymmetric,0.9099999999999999,0.394659


In [ ]:
filt = result['Beta'] == '0.01'
result.loc[filt]
filt2 = result['Alpha'] == '0.01'
result.loc[filt2]

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.185373
1,75% Corpus,2,0.01,0.31,0.220443
2,75% Corpus,2,0.01,0.61,0.207812
3,75% Corpus,2,0.01,0.9099999999999999,0.178013
4,75% Corpus,2,0.01,symmetric,0.187350
...,...,...,...,...,...
510,100% Corpus,10,0.01,0.01,0.384688
511,100% Corpus,10,0.01,0.31,0.449954
512,100% Corpus,10,0.01,0.61,0.446260
513,100% Corpus,10,0.01,0.9099999999999999,0.370303


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=20,
                                           alpha=0.61,
                                           eta=0.31)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.180937 -0.048517       1        1  14.833008
6      0.055693 -0.121555       2        1  14.558807
0      0.049501  0.042511       3        1  13.915024
4     -0.026436  0.138807       4        1  13.484529
7      0.033335 -0.020055       5        1  11.439628
3      0.012008 -0.000153       6        1  10.956627
5      0.033074  0.001482       7        1  10.872773
2      0.023762  0.007481       8        1   9.939604, topic_info=           Term        Freq       Total Category  logprob  loglift
30          это  205.000000  205.000000  Default  30.0000  30.0000
31          год  100.000000  100.000000  Default  29.0000  29.0000
303    говорить   76.000000   76.000000  Default  28.0000  28.0000
22         свой   76.000000   76.000000  Default  27.0000  27.0000
295      россия   91.000000   91.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
2591    главное    6.047101    7.376024   Topic8  -6.1381   2.1100
8220    галерея    6.010574    7.412761   Topic8  -6.1442   2.0990
617      первый   10.871450   22.906786   Topic8  -5.5516   1.5633
1027  придумать    6.101565    7.751568   Topic8  -6.1292   2.0693
541       знать    6.302958   45.445789   Topic8  -6.0967   0.3331

[315 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2330      5  0.918626       америка
1482      6  0.880452  американский
2934      5  0.879171       близкий
1563      7  0.865945     блондинка
785       8  0.893758       богатый
...     ...       ...           ...
30        2  0.937275           это
30        3  0.058276           это
484       5  0.900889          этот
1548      4  0.968202          яйцо
536       1  0.936981        япония

[306 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 1, 5, 8, 4, 6, 3])

In [ ]:
for t, top_words in lda_model.print_topics(num_topics=8, num_words=10):
  print('Topic', t+1, ':', top_words)

Topic 1 : 0.014*"сказать" + 0.011*"женщина" + 0.009*"мужик" + 0.008*"мужчина" + 0.008*"жена" + 0.007*"какой" + 0.007*"думать" + 0.007*"знать" + 0.006*"должный" + 0.006*"коррупция"
Topic 2 : 0.017*"россия" + 0.013*"человек" + 0.011*"стать" + 0.010*"деньга" + 0.007*"сам" + 0.006*"новость" + 0.006*"сенатор" + 0.006*"ребёнок" + 0.006*"который" + 0.006*"правительство"
Topic 3 : 0.027*"говорить" + 0.011*"самый" + 0.011*"ещё" + 0.009*"давать" + 0.009*"ты" + 0.008*"хороший" + 0.007*"купить" + 0.007*"начать" + 0.006*"найти" + 0.005*"дело"
Topic 4 : 0.024*"свой" + 0.009*"украина" + 0.008*"сделать" + 0.005*"сша" + 0.005*"москва" + 0.005*"интересно" + 0.004*"таки" + 0.004*"гражданин" + 0.003*"выборы" + 0.003*"имя"
Topic 5 : 0.014*"всё" + 0.013*"венесуэла" + 0.012*"президент" + 0.011*"яйцо" + 0.010*"весь" + 0.008*"мадурый" + 0.008*"путин" + 0.008*"тот" + 0.008*"каждый" + 0.007*"народ"
Topic 6 : 0.017*"хотеть" + 0.013*"время" + 0.013*"делать" + 0.010*"очень" + 0.009*"русский" + 0.008*"спрашивать" + 